In [87]:

import requests
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import time
import json
import re
from datetime import datetime
from langdetect import detect_langs


In [2]:

from selenium.webdriver.edge import service
from selenium import webdriver
from selenium.webdriver.common.by import By
from collections import Counter
from nltk.corpus import stopwords
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

## Scraping current values for GP links

In [53]:
#Load data
gp_current = pd.read_csv("data/full_gp_data.csv", sep = ";", index_col = "Unnamed: 0")

In [58]:
#gp_current[gp_current["url"].isnull()==True]

,gp_url,url,app_name,launch_date,title,main_developer_page,googleplay_link


In [59]:
gp_current

,gp_url,url,app_name,launch_date,title,main_developer_page,googleplay_link
0,play.google.com/store/apps/details?id=com.baiu...,/t/app-6-0-compass-of-chance.4440211/,Compass of Chance: Your Dream,"May 2, 2022",[APP][6.0+]Compass of Chance,/m/baiumka.12120441/,https://play.google.com/store/apps/details?id=...
1,play.google.com/store/apps/details?id=eu.fairc...,/t/app-6-0-netguard-no-root-firewall.3233012/,NetGuard - no-root firewall,"Oct 25, 2015",[APP][6.0+] NetGuard - No-root firewall,/m/m66b.2799345/,https://play.google.com/store/apps/details?id=...
2,play.google.com/store/apps/details?id=com.helg...,/t/app-11-dual-camera-video-recorder.4447581/,Dual Camera Video Recorder,Today at 8:08 AM,[APP][11+] Dual Camera Video Recorder,/m/user848.5214037/,https://play.google.com/store/apps/details?id=...
4,play.google.com/store/apps/details?id=rs.pedja...,/t/app-root-4-0-battery-charge-limit-v1-1-1.35...,[ROOT] aLogcat,"Feb 15, 2017",[APP][ROOT][4.0+]Battery Charge Limit v1.1.1,/m/sriharshaarangi.6898873/,https://play.google.com/store/apps/details?id=...
5,play.google.com/store/apps/details?id=rs.pedja...,/t/app-4-1-20140920-alogcat-root-v1-0-1.2876985/,[ROOT] aLogcat,"Sep 14, 2014",[APP][4.1+][20140920] aLogcat ROOT - v1.0.1,NaN,https://play.google.com/store/apps/details?id=...
...,...,...,...,...,...,...,...
3416,play.google.com/store/apps/details?id=com.sien...,/t/free-app-ur-launcher.2924469/,UR 3D Launcher,"Oct 31, 2014",[FREE][APP] UR launcher,/m/urlauncher.5721290/,https://play.google.com/store/apps/details?id=...
3424,play.google.com/store/apps/details?id=christma...,/t/christmas-ringtones-app.2003588/,Weihnachten Klingeltöne,"Nov 20, 2012",Christmas Ringtones [APP],/m/johnmcenroy.4949583/,https://play.google.com/store/apps/details?id=...
3425,play.google.com/store/apps/details?id=ua.maksd...,/t/app-1-6-horoscopes-d_t.1902068/,Horoscope for Tomorrow,"Sep 24, 2012",[APP][1.6+] Horoscopes D_T,/m/fuldei.4674653/,https://play.google.com/store/apps/details?id=...
3428,play.google.com/store/apps/details?id=com.alen...,/t/app-mirror-miui-gallery-and-quickpic-pretty...,Galería de fotos - QuickPic,"Apr 5, 2012",[APP-MIRROR] MIUI Gallery and QuickPic (Pretty...,/m/appkiller.4497130/,https://play.google.com/store/apps/details?id=...


In [56]:
gp_dict = gp_current.to_dict(orient="records")

In [84]:
#setting up selenium to function
edgeOption = webdriver.EdgeOptions()
edgeOption.use_chromium = True
edgeOption.add_argument("start-maximized")
edgeOption.binary_location = r"C:\Program Files (x86)\Microsoft\Edge\Application\msedge.exe"
s=service.Service(r'C:\Users\Yana Gerina\Documents\msedgedriver.exe')
driver = webdriver.Edge(service=s, options=edgeOption)

In [85]:
driver.get(gp_current.loc[55, "googleplay_link"])

In [77]:
#counter = 0
for gp_page in gp_dict[561]:
    url = gp_page["googleplay_link"]
    driver.get(url)
    time.sleep(3)
    if len(re.findall("We're sorry, the requested URL was not found on this server.", driver.page_source)) == 0:
        if len(re.findall("<div class=.+>([1-9][0-9MK]*\+)</div>", driver.page_source))>0:
            gp_page["current_demand"] = re.findall("<div class=.+>([1-9][0-9MK]*\+)</div>", driver.page_source)[0]
        gp_page["current_app_name"] = driver.find_element(By.TAG_NAME, value ="h1").text
        gp_page["current_ads"] = len(re.findall("Contains ads", driver.page_source)) > 0
        gp_page["current_paid_features"] = len(re.findall("In-app purchases", driver.page_source)) > 0
        gp_page["current_paid"] = len(re.findall('aria-label=".+ Buy">', driver.page_source)) > 0
        counter += 1
        if counter % 100 == 0:
            print(counter)

500
600
700
800
900
1000
1100
1200


In [96]:
#check language
counter=0
for gp_page in gp_dict:
    url = gp_page["googleplay_link"]
    driver.get(url)
    time.sleep(3)
    if len(re.findall("We're sorry, the requested URL was not found on this server.", driver.page_source)) == 0:
        gp_page["language_flag"]=len(re.findall("[^\[\]en0-9\.:]", str(detect_langs(driver.find_element(By.XPATH, value ="/html/body").text))))>0
    counter += 1
    if counter % 100 == 0:
        print(counter)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400


In [89]:
driver.get_text

AttributeError: 'WebDriver' object has no attribute 'text'

In [95]:
re.findall("[^en0-9\.:]", str(detect_langs(driver.find_element(By.XPATH, value ="/html/body").text)))

['[', ']']

In [100]:
finished = pd.DataFrame(gp_dict)

In [102]:
finished

,gp_url,url,app_name,launch_date,title,main_developer_page,googleplay_link,current_demand,current_app_name,current_ads,current_paid_features,current_paid,language_flag
0,play.google.com/store/apps/details?id=com.baiu...,/t/app-6-0-compass-of-chance.4440211/,Compass of Chance: Your Dream,"May 2, 2022",[APP][6.0+]Compass of Chance,/m/baiumka.12120441/,https://play.google.com/store/apps/details?id=...,10+,Compass of Chance: Your Dream,True,False,False,False
1,play.google.com/store/apps/details?id=eu.fairc...,/t/app-6-0-netguard-no-root-firewall.3233012/,NetGuard - no-root firewall,"Oct 25, 2015",[APP][6.0+] NetGuard - No-root firewall,/m/m66b.2799345/,https://play.google.com/store/apps/details?id=...,5M+,NetGuard - no-root firewall,False,True,False,False
2,play.google.com/store/apps/details?id=com.helg...,/t/app-11-dual-camera-video-recorder.4447581/,Dual Camera Video Recorder,Today at 8:08 AM,[APP][11+] Dual Camera Video Recorder,/m/user848.5214037/,https://play.google.com/store/apps/details?id=...,10+,Dual Camera Video Recorder,True,True,False,False
3,play.google.com/store/apps/details?id=rs.pedja...,/t/app-root-4-0-battery-charge-limit-v1-1-1.35...,[ROOT] aLogcat,"Feb 15, 2017",[APP][ROOT][4.0+]Battery Charge Limit v1.1.1,/m/sriharshaarangi.6898873/,https://play.google.com/store/apps/details?id=...,NaN,NaN,NaN,NaN,NaN,NaN
4,play.google.com/store/apps/details?id=rs.pedja...,/t/app-4-1-20140920-alogcat-root-v1-0-1.2876985/,[ROOT] aLogcat,"Sep 14, 2014",[APP][4.1+][20140920] aLogcat ROOT - v1.0.1,NaN,https://play.google.com/store/apps/details?id=...,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1457,play.google.com/store/apps/details?id=com.sien...,/t/free-app-ur-launcher.2924469/,UR 3D Launcher,"Oct 31, 2014",[FREE][APP] UR launcher,/m/urlauncher.5721290/,https://play.google.com/store/apps/details?id=...,1M+,UR 3D Launcher—Customize Phone,True,False,False,False
1458,play.google.com/store/apps/details?id=christma...,/t/christmas-ringtones-app.2003588/,Weihnachten Klingeltöne,"Nov 20, 2012",Christmas Ringtones [APP],/m/johnmcenroy.4949583/,https://play.google.com/store/apps/details?id=...,10K+,Christmas Ringtones,True,False,False,False
1459,play.google.com/store/apps/details?id=ua.maksd...,/t/app-1-6-horoscopes-d_t.1902068/,Horoscope for Tomorrow,"Sep 24, 2012",[APP][1.6+] Horoscopes D_T,/m/fuldei.4674653/,https://play.google.com/store/apps/details?id=...,10K+,Horoscope for Tomorrow,True,False,False,False
1460,play.google.com/store/apps/details?id=com.alen...,/t/app-mirror-miui-gallery-and-quickpic-pretty...,Galería de fotos - QuickPic,"Apr 5, 2012",[APP-MIRROR] MIUI Gallery and QuickPic (Pretty...,/m/appkiller.4497130/,https://play.google.com/store/apps/details?id=...,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
finished[finished["current_app_name"].isnull()==True]

,gp_url,url,app_name,launch_date,title,main_developer_page,googleplay_link,current_demand,current_app_name,current_ads,current_paid_features,current_paid,language_flag
3,play.google.com/store/apps/details?id=rs.pedja...,/t/app-root-4-0-battery-charge-limit-v1-1-1.35...,[ROOT] aLogcat,"Feb 15, 2017",[APP][ROOT][4.0+]Battery Charge Limit v1.1.1,/m/sriharshaarangi.6898873/,https://play.google.com/store/apps/details?id=...,NaN,NaN,NaN,NaN,NaN,NaN
4,play.google.com/store/apps/details?id=rs.pedja...,/t/app-4-1-20140920-alogcat-root-v1-0-1.2876985/,[ROOT] aLogcat,"Sep 14, 2014",[APP][4.1+][20140920] aLogcat ROOT - v1.0.1,NaN,https://play.google.com/store/apps/details?id=...,NaN,NaN,NaN,NaN,NaN,NaN
27,play.google.com/store/apps/details?id=me.kuder...,/t/app-2-1-diskinfo.1806781/,DiskInfo,"Jul 31, 2012",[APP][2.1+] DiskInfo,/m/m-q.1948032/,https://play.google.com/store/apps/details?id=...,NaN,NaN,NaN,NaN,NaN,NaN
35,play.google.com/store/apps/details?id=com.ccs....,/t/app-4-0-c-locker.2182431/,C Locker Free,"Mar 9, 2013",[APP][4.0+][C Locker],/m/astoncheah.3852997/,https://play.google.com/store/apps/details?id=...,NaN,NaN,NaN,NaN,NaN,NaN
44,play.google.com/store/apps/details?id=org.frap...,/t/app-5-0-spectrum-a-kernel-manager-for-users...,Spectrum [Root],"May 4, 2017",[APP][5.0+] Spectrum - A kernel manager for us...,/m/frap129.5051294/,https://play.google.com/store/apps/details?id=...,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1424,play.google.com/store/apps/details?id=com.smar...,/t/app-root-5-0-smartpack-kernel-manager.3854717/,SmartPack-Kernel Manager,"Oct 16, 2018",[App][Root][5.0+] SmartPack-Kernel Manager,/m/sunilpaulmathew.7060205/,https://play.google.com/store/apps/details?id=...,NaN,NaN,NaN,NaN,NaN,NaN
1435,play.google.com/store/apps/details?id=com.proj...,/t/app-2-1-my-app-list-0-7-1-01-07-2013.1609357/,My App List,"Apr 20, 2012",[APP][2.1+] My App List - 0.7.1 - 01/07/2013,/m/ffbeltran.2553221/,https://play.google.com/store/apps/details?id=...,NaN,NaN,NaN,NaN,NaN,NaN
1451,play.google.com/store/apps/details?id=com.lx.l...,/t/app-2-2-launcher8-15aug-v1-5-2.1920531/,WP Launcher (Windows Phone Style),"Oct 4, 2012",[APP][2.2+]Launcher8[15aug][v1.5.2],/m/navdeepsangwan.4586572/,https://play.google.com/store/apps/details?id=...,NaN,NaN,NaN,NaN,NaN,NaN
1456,play.google.com/store/apps/details?id=com.oxpl...,/t/app-4-1-brashpad.3031864/,Brashpad,"Feb 15, 2015",[APP][4.1+] Brashpad,/m/oxplot.4406240/,https://play.google.com/store/apps/details?id=...,NaN,NaN,NaN,NaN,NaN,NaN


In [81]:
finished[finished["current_paid"]==True]

,gp_url,url,app_name,launch_date,title,main_developer_page,googleplay_link,current_demand,current_app_name,current_ads,current_paid_features,current_paid
39,play.google.com/store/apps/details?id=com.pran...,/t/app-4-1-barquode-matrix-manager-3-4-1-08-04...,Palettes Key,"Sep 29, 2021",[APP] [4.1+] Barquode | Matrix Manager [3.4.1]...,/m/pranav-pandey.3962236/,https://play.google.com/store/apps/details?id=...,100+,Palettes Key,False,False,True
77,play.google.com/store/apps/details?id=com.bett...,/t/app-2-2-camera-block-anti-spy-protection.24...,Camera Block Pro - Anti malware & Anti spyware...,"Sep 19, 2013",[App][2.2+] Camera Block - Anti spy protection,/m/penicilin22.4985952/,https://play.google.com/store/apps/details?id=...,10K+,Camera Block Pro - Anti malware & Anti spyware...,False,False,True
88,play.google.com/store/apps/details?id=com.newi...,/t/app-5-0-just-video-player-no-bluetooth-lag-...,nPlayer,"Nov 11, 2020",[APP][5.0+] Just (Video) Player - no Bluetooth...,/m/moneytoo.426237/,https://play.google.com/store/apps/details?id=...,50K+,nPlayer,False,False,True
110,play.google.com/store/apps/details?id=com.apoz...,/t/app-4-1-contact-diary-keep-track-of-the-peo...,Contact Diary,"Sep 19, 2020",[APP][4.1+] Contact Diary - Keep track of the ...,/m/pozas.7060034/,https://play.google.com/store/apps/details?id=...,50+,Contact Diary,False,False,True
192,play.google.com/store/apps/details?id=com.mirf...,/t/app-7-0-permission-manager-x-manage-appops-...,Permission Manager X,"Nov 7, 2020",[APP][7.0+] Permission Manager X - manage AppO...,/m/mirfatif.7805402/,https://play.google.com/store/apps/details?id=...,500+,Permission Manager X,False,False,True
210,play.google.com/store/apps/details?id=com.cyge...,/t/app-2-3-root-screen-backlight-off-xda-exclu...,Screen Backlight OFF (root),"Feb 22, 2015",[APP][2.3+][ROOT] Screen Backlight OFF (xda-ex...,/m/cygery.5253144/,https://play.google.com/store/apps/details?id=...,1K+,Screen Backlight OFF (root),False,False,True
213,play.google.com/store/apps/details?id=com.hype...,/t/app-5-0-star-launcher-prime-customize-fresh...,"Star Launcher Prime - No ads, Customize, Fresh","Sep 10, 2021","[APP] [5.0+] Star Launcher Prime 🔹 Customize, ...",/m/kovachenka.11637245/,https://play.google.com/store/apps/details?id=...,50K+,Star Launcher Prime,False,False,True
240,play.google.com/store/apps/details?id=com.mral...,/t/app-5-0-free-twitter-trends-hub.3847821/,Trends Hub for Twitter,"Sep 27, 2018",[APP][5.0+][FREE] Twitter Trends Hub,/m/ashraf-alshahawy.4979875/,https://play.google.com/store/apps/details?id=...,500+,Trends Hub for Twitter,False,False,True
275,play.google.com/store/apps/details?id=com.then...,/t/app-6-0-let-me-sleep-fingerprint-timer-disa...,Let Me Sleep - Fingerprint Timer (Pro),"Apr 16, 2021",[APP][6.0+] Let me Sleep - Fingerprint Timer -...,/m/droidlover123.4728198/,https://play.google.com/store/apps/details?id=...,100+,Let Me Sleep - Fingerprint Timer (Pro),False,False,True
276,play.google.com/store/apps/details?id=com.then...,/t/app-6-0-let-me-sleep-fingerprint-timer-disa...,Let Me Sleep - Fingerprint Timer (Pro),"Jun 14, 2018",[APP][6.0+] Let me Sleep - Fingerprint Timer -...,/m/droidlover123.4728198/,https://play.google.com/store/apps/details?id=...,100+,Let Me Sleep - Fingerprint Timer (Pro),False,False,True


In [99]:
driver.quit()

In [103]:
finished.to_csv("data/current_gp_dem.csv", sep=";")